In [10]:
import pandas as pd
import numpy as np
from pandas_datareader import data, wb
from datetime import datetime
import scipy.stats as stats
import matplotlib.pyplot as plt

In [2]:
#SMA length and length of fractional differencing

In [3]:
#triple barrier method SMA

def SMA(data,profit_margin_pct, loss_tolerance_pct,SMA_len):
    timing = 1
    return_len=1
    hold = 'cash'
    acct_value = [0]*SMA_len
    untouched = [0]*SMA_len
    acct_value.append(timing)
    untouched.append(1)
    trades=[]
    trade_open = []
    trade_close = []
    SMAlist = []
    integer = 0
    
    for n in range (SMA_len, len(data)-return_len-np.mod(len(data)-SMA_len, return_len),return_len): #the stopping point maximises the loops us of dtaset without falling outside of it.
        price = data.iloc[n]
        next_price = data.iloc[n+return_len]
        SMA = data.iloc[n-SMA_len:n].mean()#-data.iloc[n-1]*.05
        SMAlist.append(SMA)
        untouched.append(untouched[-1]*next_price/price)
        acct_value.append(timing)
        integer +=1
        
        if (price >= SMA) & (hold =='cash'):#buy signal
            hold = 'stock'
            open_position = price
            trade_open.append(data.index[n])
            timing = next_price/price*timing
            integer = 0
            continue
        
        elif (hold=='stock'):
            if (price >= (1+profit_margin_pct)*open_position) & (hold =='stock'):# profit sell signal
                hold ='cash'
                close_position = price
                trades.append(2)
                trade_close.append(data.index[n])
                continue
       
            elif (price < (1-loss_tolerance_pct)*open_position) & (hold =='stock'):# loss sell signal
                hold ='cash'
                close_position = price
                trades.append(0)
                trade_close.append(data.index[n])
                continue
             
            elif (integer == SMA_len) & (hold =='stock'): # neither
                hold ='cash'
                close_position = price
                trades.append(1)
                trade_close.append(data.index[n])
                continue
            
            elif (hold =='stock'): #hold signal
                timing = next_price/price*timing
                continue 
       
    return(timing, acct_value, trades, trade_open, trade_close, SMAlist, untouched)

In [4]:
def getWeights(d,lags):
    # return the weights from the series expansion of the differencing operator
    # for real orders d and up to lags coefficients
    w=[1]
    for k in range(1,lags):
        w.append(-w[-1]*((d-k+1))/k)
    w=np.array(w).reshape(-1,1) 
    return w

def ts_differencing(series, order, lag_cutoff):
    # return the time series resulting from (fractional) differencing
    # for real orders order up to lag_cutoff coefficients
    
    weights=getWeights(order, lag_cutoff)
    res=0
    for k in range(lag_cutoff):
        res += weights[k]*series.shift(k).fillna(0)
    return res[lag_cutoff:] 

In [5]:
#from_https://towardsdatascience.com/easily-visualize-scikit-learn-models-decision-boundaries-dd0fb3747508

#change this so you can select the columns of X

In [9]:
def plot_decision_boundaries(X, X1, X2, y, model_class, **model_params):
    """
    Function to plot the decision boundaries of a classification model.
    This uses just two columns of the data for fitting 
    the model as we need to find the predicted value for every point in 
    scatter plot.
    Arguments:
            X: Feature data as a NumPy-type array.
            y: Label data as a NumPy-type array.
            model_class: A Scikit-learn ML estimator class 
            e.g. GaussianNB (imported from sklearn.naive_bayes) or
            LogisticRegression (imported from sklearn.linear_model)
            **model_params: Model parameters to be passed on to the ML estimator
    
    Typical code example:
            plt.figure()
            plt.title("KNN decision boundary with neighbros: 5",fontsize=16)
            plot_decision_boundaries(X_train,y_train,KNeighborsClassifier,n_neighbors=5)
            plt.show()
    """
    try:
        X = np.array(X)
        y = np.array(y).flatten()
    except:
        print("Coercing input data to NumPy arrays failed")
    # Reduces to the first two columns of data
    #reduced_data = X[:, :2]
    col1 = np.reshape(X[:,X1],[-1,1])
    col2 = np.reshape(X[:,X2],[-1,1])
    reduced_data = np.concatenate([col1,col2],axis=1)
    
    # Instantiate the model object
    model = model_class(**model_params)
    # Fits the model with the reduced data
    model.fit(reduced_data, y)

    # Step size of the mesh. Decrease to increase the quality of the VQ.
    h = .02     # point in the mesh [x_min, m_max]x[y_min, y_max].    

    # Plot the decision boundary. For that, we will assign a color to each
    x_min, x_max = reduced_data[:, 0].min() - 1, reduced_data[:, 0].max() + 1
    y_min, y_max = reduced_data[:, 1].min() - 1, reduced_data[:, 1].max() + 1
    # Meshgrid creation
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))

    # Obtain labels for each point in mesh using the model.
    Z = model.predict(np.c_[xx.ravel(), yy.ravel()])    

    x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.1),
                         np.arange(y_min, y_max, 0.1))

    # Predictions to obtain the classification results
    Z = model.predict(np.c_[xx.ravel(), yy.ravel()]).reshape(xx.shape)

    # Plotting
    plt.contourf(xx, yy, Z, alpha=0.1)
    plt.scatter(X[:, X1], X[:, X2], c=y, alpha=0.8)   #plt.scatter(X[:, 0], X[:, 1], c=y, alpha=0.8)
    plt.xlabel("X",fontsize=15)
    plt.ylabel("Y",fontsize=15)
    plt.xticks(fontsize=14)
    plt.yticks(fontsize=14)
    return plt

In [7]:
#triple barrier method SMA with SVC to size bets
#got code working on 2015

def SMA_w_SVCmeta_labeling(data, profit_margin_pct, loss_tolerance_pct, SMA_len, features,SVC_model):
    timing = 1
    return_len=1
    hold = 'cash'
    acct_value = [0]*SMA_len
    acct_value.append(timing)
    trades=[]
    trade_open = []
    trade_close = []
    SMAlist = []
    integer = 0
    
    for n in range (SMA_len, len(data)-return_len-np.mod(len(data)-SMA_len, return_len),return_len): #the stopping point maximises the loops us of dtaset without falling outside of it.
        price = data.iloc[n]
        next_price = data.iloc[n+return_len]
        SMA = data.iloc[n-SMA_len:n].mean()
        SMAlist.append(SMA)
        acct_value.append(timing)
        integer +=1
        
        if (price >= SMA) & (hold =='cash'):#buy signal
            if (SVC_model.predict(np.reshape(features[n],[1,-1]))[0] != 0):    
                hold = 'stock'
                open_position = price
                trade_open.append(data.index[n])
                timing = next_price/price*timing
                integer = 0
                continue
        
        elif (hold=='stock'):
            if (price >= (1+profit_margin_pct)*open_position) & (hold =='stock'):# profit sell signal
                hold ='cash'
                close_position = price
                trades.append(2)
                trade_close.append(data.index[n])
                continue
       
            elif (price < (1-loss_tolerance_pct)*open_position) & (hold =='stock'):# loss sell signal
                hold ='cash'
                close_position = price
                trades.append(0)
                trade_close.append(data.index[n])
                continue
             
            elif (integer == SMA_len) & (hold =='stock'): # neither
                hold ='cash'
                close_position = price
                trades.append(1)
                trade_close.append(data.index[n])
                continue
            
            elif (hold =='stock'): #hold signal
                timing = next_price/price*timing
                continue 
       
    return(timing, acct_value, trades, trade_open, trade_close, SMAlist)